In [1]:
# Problem 2: Model Compression
# 1.Compress your new model from problem 1 by converting it into a TF Lite model.
# 2.Use the TensorFlow Interpreter to load your compressed model in (1) and classify new images.
# 3.How big is your compressed model compared to the pre-compressed model in problem 1?
# 4.How quickly does your compressed model classify an image?
# 5.How much did compressing the model affect classification accuracy? 

In [1]:
import os
import tensorflow as tf
import numpy as np
import time
from PIL import Image

In [2]:
data = []
labels = []
labels = []
classes = 43
cur_path = os.getcwd()
data_directory = './archive'

for i in range(classes):
    path = os.path.join(data_directory, 'Train', str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(os.path.join(path, a))
            image = image.resize((30, 30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

data = np.array(data)
data = tf.data.Dataset.from_tensor_slices(data)
data = data.shuffle(buffer_size=len(data))

### Step 1

In [4]:
# Convert the model (let's use the saved model from the previous exercise)
import_path = os.path.join(os.getcwd(),'saved_models','model')
converter = tf.lite.TFLiteConverter.from_saved_model(import_path) # path to the SavedModel directory

In [9]:
#converter.allow_custom_ops = True
def representative_data_gen():
  for input_value in data.batch(1).take(1):
    # Model has only one input so each data point has one element.
    yield [input_value]

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS,  # Enable TensorFlow ops.
]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

# Let's create the folder "saved_tflite_models" and store the TFLite models there
os.makedirs(os.path.join(os.getcwd(),'saved_tflite_models'), exist_ok = True)

# Save the model
tflite_model_path = os.path.join(os.getcwd(),'saved_tflite_models','model.tflite')
with open(tflite_model_path, 'wb') as f:
  f.write(tflite_model)

ValueError: Cannot set tensor: Got value of type UINT8 but expected type FLOAT32 for input 0, name: serving_default_keras_layer_input:0 

### Step 2

In [5]:
# Classify all test ds here

### Step 3

In [9]:
def getfoldersize(path):
  total_size = 0
  for dirpath, dirnames, filenames in os.walk(path):
      for f in filenames:
          fp = os.path.join(dirpath, f)
          # skip if it is symbolic link
          if not os.path.islink(fp):
              total_size += os.path.getsize(fp)

  total_size /= (1024 ** 3)
  total_size = round(total_size, 3)
  return total_size

old_model_size = getfoldersize(os.path.join(os.getcwd(),'saved_models','model'))
new_model_size = getfoldersize(os.path.join(os.getcwd(),'saved_tflite_models'))

print("GB: " + str(old_model_size))
print("GB: " + str(new_model_size))

GB: 1.468
GB: 1.429


### Step 4

In [7]:
start = time.time()

# Classify 1 image here

end = time.time()
print(end - start)

0.0


### Step 5

In [8]:
# Compere old model and new model acc here (need to load and deload old model)